In [133]:
# import libraries
import FinanceDataReader as fdr
import numpy as np
from datetime import datetime, timedelta

In [134]:
# 데이터 수집 기간 및 종목코드 설정
start_date = '20140106'
end_date = '20220527'
code = '000060' # test code

In [135]:
# 시작일이 주말인 경우: 예외처리
start_datetime = datetime.strptime(start_date, '%Y%m%d')
if (start_datetime.weekday() == 5): # 시작: 토요일
    start_datetime += timedelta(days=2)
    start_date = start_datetime.strftime('%Y%m%d')
elif (start_datetime.weekday() == 6): # 시작: 일요일
    start_datetime += timedelta(days=1)
    start_date = start_datetime.strftime('%Y%m%d')

In [136]:
# start_data를 조정하여 ma 계산에 필요한 여분 데이터 확보
new_start_datetime = start_datetime - timedelta(weeks=14)
new_start_date = new_start_datetime.strftime('%Y%m%d')

In [137]:
# 종목코드에 대한 종가 데이터 수집
data = fdr.DataReader(code, new_start_date, end_date).reset_index()
data.drop(['Open', 'High', 'Low', 'Change', 'Volume'], axis=1, inplace=True)

# 5/20/60일 이동평균 계산
ma5 = data['Close'].rolling(window=5).mean()
data['ma5'] = ma5
ma20 = data['Close'].rolling(window=20).mean()
data['ma20'] = ma20
ma60 = data['Close'].rolling(window=60).mean()
data['ma60'] = ma60

# 이동평균 변화율(Change) 계산
data['ma5_Change'] = np.ceil((data['ma5']/data['ma5'].shift(1) - 1) * 100 * 10000)/10000
data['ma20_Change'] = np.ceil((data['ma20']/data['ma20'].shift(1) - 1) * 100 * 10000)/10000
data['ma60_Change'] = np.ceil((data['ma60']/data['ma60'].shift(1) - 1) * 100 * 10000)/10000

# Nan 데이터 제거
for idx in data.index:
    if (data.loc[idx, 'Date'].to_pydatetime().strftime('%Y%m%d') != start_date):
        data.drop(idx, inplace=True)
    else:
        break
data.reset_index(inplace=True)
data.drop(['index'], axis=1, inplace=True)

# 이동평균 데이터타입 변환 (float to int)
data['ma5'] = data['ma5'].astype(int)
data['ma20'] = data['ma20'].astype(int)
data['ma60'] = data['ma60'].astype(int)

print(data)

           Date  Close    ma5   ma20   ma60  ma5_Change  ma20_Change  \
0    2014-01-06  15350  15340  15067  14720     -0.5188      -0.0165   
1    2014-01-07  15400  15360  15070  14750      0.1304       0.0166   
2    2014-01-08  15400  15410  15090  14780      0.3256       0.1328   
3    2014-01-09  15600  15430  15130  14815      0.1298       0.2651   
4    2014-01-10  15500  15450  15160  14842      0.1297       0.1983   
...         ...    ...    ...    ...    ...         ...          ...   
2060 2022-05-23  39750  39170  41387  42754      1.2407      -0.6541   
2061 2022-05-24  39700  39460  41150  42719      0.7404      -0.5738   
2062 2022-05-25  40800  39670  40945  42682      0.5322      -0.4981   
2063 2022-05-26  39800  39830  40700  42660      0.4034      -0.5983   
2064 2022-05-27  39800  39970  40442  42679      0.3515      -0.6326   

      ma60_Change  
0          0.1361  
1          0.1982  
2          0.2034  
3          0.2369  
4          0.1857  
...           .